In [1]:
import numpy as np
import pandas as pd
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from itertools import product
from datetime import datetime
import random

# my own library
from backTester.BackTester import BackTester
from backTester.BackDataLoader import BackDataLoader
from conditionGenerator.ConditionGenerator import ConditionGenerator

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
## 아래 꺼가. GridSearch하는 방법

params = {
    'windowMA':[20, 25, 30],
    'windowER':[15, 20, 25],
    # 'c_std':[1.5, 1.65, 1.8, 2, 2.15],
    'c_ERmean':[1.75, 2],
    'pastFirst':[8, 10],
    'pastLast':[13, 15],
    'nowFirst':[1, 3],
    'nowLast':[4, 6],
    'th_Volume':[20000, 25000, 30000],
    'volFirst':[7, 11],
    'volLast':[14, 18]

}

items = list(params.values())
keys = list(params.keys())
prods = list(product(*items))
print(len(prods))

random.seed(777)
samples = random.sample(prods, len(prods)//20)
print(len(samples))

3456
172


In [3]:
resultNum = 20
combIndexStorer = [-1]*resultNum
paramStorer = [0]*resultNum
max_n_results = [1]*resultNum

monthPair = [[1, 2], [2, 3], [3, 4], [4, 5], [7, 8], [8, 9], [9, 10]]
backdatas = dict()
for i, (m1, m2) in enumerate(monthPair):
    backdatas[i] = pd.read_pickle(f"BackDataList\\backdata1m_period_{m1}_1_to_{m2}_1.pkl")

for combIndex, comb in enumerate(samples):
    returnList = []

    print('combIndex: ', combIndex, end=' ')
    param = dict()
    for key, value in zip(keys, comb):
        param[key] = value
        # 요기서부터 집어넣는다.
    print('Param:', list(param.values()), end=' ')

    for i in range(7):
        backdata = backdatas[i]
    
        tester = BackTester()
        # print(id(tester), end='')

        window_MA = param['windowMA']
        window_ER = param['windowER']
        backdata['MA'] = tester.get_MA(window_MA, backdata['Close'])
        backdata['std'] = tester.get_std(window_MA, backdata['Close'])
        backdata['UB'] = backdata['MA'] + 2 * backdata['std']
        backdata['U75'] = backdata['MA'] + backdata['std']
        backdata['LB'] = backdata['MA'] - 2 * backdata['std']
        backdata['L25'] = backdata['MA'] - backdata['std']
        backdata['ER'] = backdata['Close'].diff(window_ER).abs()/backdata['Close'].diff().abs().rolling(window_ER).sum()

        backdata['ERmean'] = backdata['ER'].mean()*param['c_ERmean']

        myStrg = ConditionGenerator(19, 0) # i-4, i-3, i-2, i-1, i


        myStrg.add_andCondition('Close', 'LB', '>', [param['pastFirst'], param['pastLast']])
        myStrg.add_andCondition('Close', 'LB', '<', [param['nowFirst'], param['nowLast']])
        myStrg.add_andCondition('Volume', param['th_Volume'], '<', [param['volFirst'], param['volLast']])
        myStrg.add_andCondition('ER', 'ERmean', '<', [param['volFirst'], param['volLast']])
        myStrg.add_condition('long')

        myStrg.add_andCondition('Close', 'UB', '<', [param['pastFirst'], param['pastLast']])
        myStrg.add_andCondition('Close', 'UB', '>', [param['nowFirst'], param['nowLast']])
        myStrg.add_andCondition('Volume', param['th_Volume'], '<', [param['volFirst'], param['volLast']])
        myStrg.add_andCondition('ER', 'ERmean', '<', [param['volFirst'], param['volLast']])
        myStrg.add_condition('short')

        myStrg.add_andCondition('Close', 'MA', '<', [3, 5])
        myStrg.add_andCondition('Close', 'MA', '>', [0, 2])
        myStrg.add_andCondition('_long_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', '_long_meanPrice', '<', 0, func2=lambda x:x*(1-0.03))
        myStrg.add_andCondition('_long_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', 'MA', '>', [3, 5])
        myStrg.add_andCondition('Close', 'MA', '<', [0, 2])
        myStrg.add_andCondition('_short_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', '_short_meanPrice', '>', 0, func2=lambda x:x*(1+0.03))
        myStrg.add_andCondition('_short_flag', True, '==', 0)
        myStrg.add_condition('clear')

        # backtest
        tester.backtest_tmp(myStrg, backdata, printLog=False)

        # r = tester.finalReturn
        returnList.append(tester.finalReturn)
        print(round(tester.finalReturn, 3), end=' ')
    r = sum(returnList)/7
    std = np.std(returnList)
    for resIdx in range(resultNum):
        if r > max_n_results[resIdx]:
            try:
                max_n_results = max_n_results[:resIdx] + [r] + max_n_results[resIdx:resultNum-1]
                combIndexStorer = combIndexStorer[:resIdx] + [combIndex] + combIndexStorer[resIdx:resultNum-1]
                paramStorer = paramStorer[:resIdx] + [comb] + paramStorer[resIdx:resultNum-1]
            except:
                max_n_results = [r] + max_n_results[resIdx:resultNum-1]
                combIndexStorer = [combIndex] + combIndexStorer[resIdx:resultNum-1]
                paramStorer = [comb] + paramStorer[resIdx:resultNum-1]

        # print(', Return:', r)
        
    print('std:', std, ' Return:', r)

with open('BB_GridSearch_params.txt', 'a') as f:
    f.write(str(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))+'\n'+'-'*20)
    f.write('Keys:'+str(keys))
    for idx, res, param in zip(combIndexStorer, max_n_results, paramStorer):
        f.write('Index'+str(idx)+' - Return - '+str(res)+' - Param '+str(param))


combIndex:  0 Param: [20, 25, 1.75, 10, 15, 3, 4, 20000, 11, 18] 0.962 1.003 1.005 1.077 1.054 0.977 0.982 std: 0.03914195581372122  Return: 1.0085862158571433
combIndex:  1 Param: [25, 20, 2, 10, 13, 1, 4, 20000, 11, 14] 1.033 1.008 1.071 1.055 1.045 0.997 0.996 std: 0.027422090134997683  Return: 1.0292182894285717
combIndex:  2 Param: [25, 20, 2, 10, 13, 1, 4, 20000, 7, 18] 1.019 1.002 1.078 1.052 1.021 0.981 1.008 std: 0.030097257669524483  Return: 1.0229499934285717
combIndex:  3 Param: [25, 15, 2, 10, 15, 1, 6, 30000, 7, 18] 0.999 1.01 1.006 0.991 0.984 0.988 0.991 std: 0.00910957349817084  Return: 0.9956594501428572
combIndex:  4 Param: [30, 15, 1.75, 8, 15, 1, 4, 25000, 7, 18] 0.987 1.032 1.012 1.012 1.039 1.016 0.984 std: 0.01899651667246842  Return: 1.0118372069999997
combIndex:  5 Param: [20, 25, 2, 10, 15, 1, 4, 20000, 7, 18] 0.98 1.008 1.019 1.03 1.038 0.989 0.992 std: 0.020526597771805627  Return: 1.0078222521428575
combIndex:  6 Param: [30, 25, 2, 10, 13, 3, 6, 30000, 7, 

In [6]:
paramStorer

[(25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14),
 (25, 20, 2, 10, 13, 1, 4, 20000, 11, 14)]

In [ ]:
with open('Results221019_1m.txt', 'r') as f:
    f.readline(a)

In [10]:
## 아래 꺼가. GridSearch하는 방법

params = {
    'windowMA':[20, 25, 30],
    'windowER':[15, 20, 25],
    # 'c_std':[1.5, 1.65, 1.8, 2, 2.15],
    'c_ERmean':[1.75, 2],
    'pastFirst':[8, 10],
    'pastLast':[13, 15],
    'nowFirst':[1, 3],
    'nowLast':[4, 6],
    'volFirst':[7, 11],
    'volLast':[14, 18]

}

items = list(params.values())
keys = list(params.keys())
prods = list(product(*items))
print(len(prods))

random.seed(777)
samples = random.sample(prods, len(prods)//10)
print(len(samples))

1152
115


In [11]:
resultNum = 20
combIndexStorer = [-1]*resultNum
paramStorer = [0]*resultNum
max_n_results = [1]*resultNum

monthPair = [[1, 2], [2, 3], [3, 4], [4, 5], [7, 8], [8, 9], [9, 10]]
backdatas = dict()
for i in range(7):
    backdatas[i] = pd.read_pickle(f"BackDataList\\backdata{i}.pkl")

for combIndex, comb in enumerate(samples):
    returnList = []

    print('combIndex: ', combIndex, end=' ')
    param = dict()
    for key, value in zip(keys, comb):
        param[key] = value
        # 요기서부터 집어넣는다.
    print('Param:', list(param.values()), end=' ')

    for i in range(7):
        backdata = backdatas[i]
    
        tester = BackTester()
        # print(id(tester), end='')

        window_MA = param['windowMA']
        window_ER = param['windowER']
        backdata['MA'] = tester.get_MA(window_MA, backdata['Close'])
        backdata['std'] = tester.get_std(window_MA, backdata['Close'])
        backdata['UB'] = backdata['MA'] + 2 * backdata['std']
        backdata['U75'] = backdata['MA'] + backdata['std']
        backdata['LB'] = backdata['MA'] - 2 * backdata['std']
        backdata['L25'] = backdata['MA'] - backdata['std']
        backdata['ER'] = backdata['Close'].diff(window_ER).abs()/backdata['Close'].diff().abs().rolling(window_ER).sum()

        backdata['ERmean'] = backdata['ER'].mean()*param['c_ERmean']

        myStrg = ConditionGenerator(19, 0) # i-4, i-3, i-2, i-1, i


        myStrg.add_andCondition('Close', 'LB', '>', [param['pastFirst'], param['pastLast']])
        myStrg.add_andCondition('Close', 'LB', '<', [param['nowFirst'], param['nowLast']])
        # myStrg.add_andCondition('Volume', param['th_Volume'], '<', [param['volFirst'], param['volLast']])
        myStrg.add_andCondition('ER', 'ERmean', '<', [param['volFirst'], param['volLast']])
        myStrg.add_condition('long')

        myStrg.add_andCondition('Close', 'UB', '<', [param['pastFirst'], param['pastLast']])
        myStrg.add_andCondition('Close', 'UB', '>', [param['nowFirst'], param['nowLast']])
        # myStrg.add_andCondition('Volume', param['th_Volume'], '<', [param['volFirst'], param['volLast']])
        myStrg.add_andCondition('ER', 'ERmean', '<', [param['volFirst'], param['volLast']])
        myStrg.add_condition('short')

        myStrg.add_andCondition('Close', 'MA', '<', [3, 5])
        myStrg.add_andCondition('Close', 'MA', '>', [0, 2])
        myStrg.add_andCondition('_long_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', '_long_meanPrice', '<', 0, func2=lambda x:x*(1-0.03))
        myStrg.add_andCondition('_long_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', 'MA', '>', [3, 5])
        myStrg.add_andCondition('Close', 'MA', '<', [0, 2])
        myStrg.add_andCondition('_short_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', '_short_meanPrice', '>', 0, func2=lambda x:x*(1+0.03))
        myStrg.add_andCondition('_short_flag', True, '==', 0)
        myStrg.add_condition('clear')

        # backtest
        tester.backtest_tmp(myStrg, backdata, printLog=False)

        # r = tester.finalReturn
        returnList.append(tester.finalReturn)
        print(round(tester.finalReturn, 3), end=' ')
    r = sum(returnList)/7
    std = np.std(returnList)
    for resIdx in range(resultNum):
        if r > max_n_results[resIdx]:
            try:
                max_n_results = max_n_results[:resIdx] + [r] + max_n_results[resIdx:resultNum-1]
                combIndexStorer = combIndexStorer[:resIdx] + [combIndex] + combIndexStorer[resIdx:resultNum-1]
                paramStorer = paramStorer[:resIdx] + [comb] + paramStorer[resIdx:resultNum-1]
            except:
                max_n_results = [r] + max_n_results[resIdx:resultNum-1]
                combIndexStorer = [combIndex] + combIndexStorer[resIdx:resultNum-1]
                paramStorer = [comb] + paramStorer[resIdx:resultNum-1]

        # print(', Return:', r)
        
    print('std:', std, ' Return:', r)

with open('BB_GridSearch_params.txt', 'a') as f:
    f.write(str(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))+'\n'+'-'*20)
    f.write('Keys:'+str(keys))
    for idx, res, param in zip(combIndexStorer, max_n_results, paramStorer):
        f.write('Index'+str(idx)+' - Return - '+str(res)+' - Param '+str(param))


combIndex:  0 Param: [25, 15, 2, 8, 15, 1, 6, 7, 18] 0.974 1.123 0.993 1.007 0.99 0.99 1.005 std: 0.04645319200620735  Return: 1.0117085118571427
combIndex:  1 Param: [30, 20, 1.75, 8, 15, 1, 4, 7, 18] 0.97 1.019 1.033 1.028 1.017 0.975 0.975 std: 0.02571156559670092  Return: 1.0022350958571427
combIndex:  2 Param: [30, 20, 1.75, 8, 15, 1, 4, 7, 14] 0.961 1.011 1.027 1.024 1.01 0.974 0.966 std: 0.02586929930481436  Return: 0.9961034894285711
combIndex:  3 Param: [25, 25, 2, 10, 15, 1, 4, 11, 14] 0.997 1.097 1.009 1.018 1.008 0.979 0.989 std: 0.0359018845131926  Return: 1.0140338334285715
combIndex:  4 Param: [25, 20, 1.75, 10, 13, 3, 4, 7, 14] 1.001 1.108 0.999 0.997 0.966 0.971 1.005 std: 0.04353994391449737  Return: 1.0067672204285718
combIndex:  5 Param: [25, 25, 1.75, 10, 15, 1, 4, 7, 14] 1.008 1.083 1.018 1.015 1.001 0.984 0.971 std: 0.03314971945424313  Return: 1.0113512251428574
combIndex:  6 Param: [30, 25, 2, 8, 13, 3, 6, 7, 18] 0.97 1.025 1.004 0.997 1.012 0.975 0.973 std: 0.